# Rechtspraak, LI & citations to DynamoDB ETL

In [ ]:
!pip install boto3

In [1]:
import boto3
from boto3.dynamodb import conditions
from boto3.dynamodb.conditions import Key, Attr
from decimal import Decimal
import json
import csv
from pprint import pprint
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import string
from datetime import datetime
import dateutil.parser

- rs = rechtspraak data
- li = legal intelligence data
---
## Raw Datasets

In [2]:
pd.set_option('display.max_columns', None)
df_rs_cases_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case.csv", dtype=str).replace({np.nan: None})  #, nrows=500000).replace(np.nan, None, regex=True)
df_rs_cases_raw.head()

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,subject,authority,legal_references,related_cases,alternative_sources,full_textcase_id,date.1,case_number.1,description.1,language.1,venue.1,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:GHAMS:1913:15,1913-03-31,Onbekend,None,NL,None,None,None,None,NL,None,Gerechtshof Amsterdam,None,None,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ECLI:NL:GHAMS:1913:28,1913-02-28,Onbekend389,None,NL,None,None,None,None,NL,None,Gerechtshof Amsterdam,None,None,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ECLI:NL:GHAMS:1913:19,1913-03-17,Onbekend,None,NL,None,None,None,None,NL,None,Gerechtshof Amsterdam,None,None,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ECLI:NL:GHAMS:1913:4,1913-02-28,Onbekend,None,NL,None,None,None,None,NL,None,Gerechtshof Amsterdam,None,None,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ECLI:NL:GHAMS:1913:20,1913-03-10,Onbekend,None,NL,None,None,None,None,NL,None,Gerechtshof Amsterdam,None,None,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [3]:
df_rs_opinions_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case_opinion_from_advocate_general.csv", dtype='str').replace({np.nan: None})
df_rs_opinions_raw.head()

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,subject,authority,legal_references,related_cases,alternative_sources,full_textcase_id,date.1,case_number.1,description.1,language.1,venue.1,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:PHR:1913:AG1763,1913-04-18,x47,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1763\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ECLI:NL:PHR:1913:AG1764,1913-04-18,x48,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1764\n,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ECLI:NL:PHR:1913:BG9429,1913-04-21,None,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:BG9429\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ECLI:NL:PHR:1913:AG1832,1913-03-07,X46,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1832\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ECLI:NL:PHR:1913:AG1762,1913-06-20,x49,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1762\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [4]:
df_li_cases_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legal_intelligence_cases.csv", dtype=str).replace({np.nan: None}).drop_duplicates(subset ="ecli")
df_li_cases_raw.head()

,Unnamed: 0,IssueNumber,Id,Title,DisplayTitle,DisplaySubtitle,abstract,Url,OriginalUrl,Jurisdiction,DocumentType,authority,case_number,date,lodge_date,DateAdded,Sources,UrlWithAutoLogOnToken,LawArea,ecli,PublicationNumber
0,0,None,4968258,Casus Bij besluit van 12 april 1994 verlenen g...,Casus Bij besluit van 12 april 1994 verlenen g...,"Doorduijn, L.F. - Milieurecht Totaal - Module-...",Casus Bij besluit van 12 april 1994 verlenen g...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da785f3d8d...,Nederland,Rechtspraak,Raad van State,E03951413,19960101,19960916,20110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_AP7727,None
1,1,None,4968759,M.N. Boeve Casus GS van Noord-Brabant hebben (...,M.N. Boeve Casus GS van Noord-Brabant hebben (...,"Boeve, M.N. - Milieurecht Totaal - Module-seri...",M.N. Boeve Casus GS van Noord-Brabant hebben (...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dae9667789...,Nederland,Rechtspraak,Raad van State,E01.94.0417 ECLI:NL:RVS:1996:AP7883,19960101,19960708,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_AP7883,None
2,2,None,4968382,Naast woning gelegen huisje vormt ┌┌n geheel m...,Naast woning gelegen huisje vormt ┌┌n geheel m...,Redactie - Module-serie - Module Afval - Milie...,Naast woning gelegen huisje vormt ┌┌n geheel m...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da7b16fb16...,Nederland,Rechtspraak,Gerechtshof Amsterdam,95/4927-E,19960101,19961203,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_GHAMS_1996_AS4281,None
3,3,None,4968678,Geen beroep mogelijk tegen weigering bestemmin...,Geen beroep mogelijk tegen weigering bestemmin...,Redactie - Module-serie - Module Ruimtelijke O...,De raad van de gemeente Waspik wees een verzoe...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dad97422ad...,Nederland,Rechtspraak,Raad van State,H01950354,19960101,19960201,20110628,Module Ruimtelijke Ordening,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_ZF1978,None
4,4,None,4968777,Bodemverontreiniging; saneringskosten komen oo...,Bodemverontreiniging; saneringskosten komen oo...,"Kruimel, J.P. - Milieurecht Totaal",Bodemverontreiniging; saneringskosten komen oo...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/daf99e49b6...,Nederland,Rechtspraak,Gerechtshof Den Haag,95/1803,19960101,19961016,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_GHSGR_1996_AS3490,None


In [7]:
df_rs_case_citations_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/caselaw_citations.csv").replace({np.nan: None})
df_rs_case_citations_raw.head()

,source_ecli,source_paragraph,target_ecli,target_paragraph
0,ECLI:NL:HR:1913:13,NULL,ECLI:NL:HR:1913:129,NULL
1,ECLI:NL:HR:1913:130,NULL,ECLI:NL:HR:1913:129,NULL
2,ECLI:NL:HR:1913:131,NULL,ECLI:NL:HR:1913:129,NULL
3,ECLI:NL:HR:1913:132,NULL,ECLI:NL:HR:1913:129,NULL
4,ECLI:NL:HR:1913:133,NULL,ECLI:NL:HR:1913:129,NULL


In [9]:
df_rs_legislation_citations_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legislation_citations.csv").replace({np.nan: None})
df_rs_legislation_citations_raw.head()

,source_ecli,source_paragraph,target_article,target_article_paragraph,target_article_webpage
0,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...
1,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...
2,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2018-...
3,ECLI:NL:GHAMS:1962:AX8441,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...
4,ECLI:NL:GHAMS:1962:AX8441,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...


In [5]:
df_country_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/countries.csv").replace({np.nan: None})
df_country_raw.head()

,country_id,eu,name,flag
0,AT,1,Austria,https://www.countryflags.io/at/flat/64.png
1,BE,1,Belgium,https://www.countryflags.io/be/flat/64.png
2,BG,1,Bulgaria,https://www.countryflags.io/bg/flat/64.png
3,CH,0,Switzerland,https://www.countryflags.io/ch/flat/64.png
4,CZ,1,Czechia,https://www.countryflags.io/cz/flat/64.png


#### stats raw data

In [6]:
print('number of rs-cases: ', len(df_rs_cases_raw))
print('number of opinions: ', len(df_rs_opinions_raw))
print('number of li-cases: ', len(df_li_cases_raw))
print('number of legislation citations: ', len(df_rs_legislation_citations_raw))
print('number of case citations: ', len(df_rs_case_citations_raw))
print('number of countries: ', len(df_country_raw))

number of rs-cases:  2632420
number of opinions:  65692
number of li-cases:  661


NameError: name 'df_rs_legislation_citations_raw' is not defined

---
## Utils

In [7]:
def clean_strings(column):
    cleaned = []
    for s in column:
        if s is not None:
            cleaned.append("".join(filter(lambda char: char in string.printable, s)))
        else:
            cleaned.append("")
    return [i[0:250] for i in cleaned]

---
## Transformation

Unify eclis:

In [8]:
for i, li_case in df_li_cases_raw.iterrows():
    ecli = li_case.ecli
    new_ecli = ecli.replace('_', ':')
    df_li_cases_raw.at[i, 'ecli'] = new_ecli

#### RS cases

In [9]:
map_rs = {'case_id':'ecli',
          'date':'date',
          'subject':'subject',
          'authority':'court',
          'case_number':'case_number',
          'description':'description',
          'language':'language',
          'country':'country',
          'venue':'venue',
          'abstract':'abstract',
          'procedure_type':'procedure_type',
          'lodge_date':'lodge_date',
          'alternative_sources':'link'}
df_rs_cases = df_rs_cases_raw[map_rs.keys()].rename(columns=map_rs)
df_rs_cases['name'] = None
df_rs_cases['doctype'] = 'uitspraak'

# clean strings and transform date type
df_rs_cases['date'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_rs_cases['date']]
df_rs_cases['lodge_date'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_rs_cases['lodge_date']]
df_rs_cases['description'] = clean_strings(df_rs_cases['description'])
df_rs_cases['link'] = clean_strings(df_rs_cases['link'])
df_rs_cases['subject'] = df_rs_cases['subject'].apply(lambda x: set(x.split("; ")) if x is not None else x)

#### RS opinions

In [10]:
df_rs_opinions = df_rs_opinions_raw[map_rs.keys()].rename(columns=map_rs)
df_rs_opinions['doctype'] = 'opinion'

df_rs_opinions['date'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_rs_opinions['date']]
df_rs_opinions['abstract'] = clean_strings(df_rs_opinions['abstract'])
df_rs_opinions['subject'] = df_rs_opinions['subject'].apply(lambda x: set(x.split("; ")) if x is not None else x)

#### LI cases

In [11]:
map_li = {
    'ecli':'ecli',
    'Title':'name_li',
    'date':'date_li',
    'abstract':'abstract_li',
    'LawArea':'subject_li',
    'authority':'court_li',
    'Url':'link_li',
    'DisplayTitle':'DisplayTitle_li',
    'OriginalUrl':'OriginalUrl_li',
    'Jurisdiction':'Jurisdiction_li',
    'DocumentType':'DocumentType_li',
    'case_number':'CaseNumber_li',
    'PublicationNumber':'PublicationNumber_li',
    'IssueNumber':'IssueNumber_li',
    'lodge_date':'lodge_date_li',
    'DateAdded':'DateAdded_li',
    'Sources':'Sources_li',
    'UrlWithAutoLogOnToken':'UrlWithAutoLogOnToken_li',
    'DisplaySubtitle':'DisplaySubtitle_li'
}
df_li_cases = df_li_cases_raw[map_li.keys()].rename(columns=map_li)
df_li_cases['doctype'] = 'uitspraak'

# clean strings and transform date type
df_li_cases['date_li'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_li_cases_raw['date']]
df_li_cases['lodge_date_li'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_li_cases_raw['lodge_date']]
df_li_cases['DateAdded_li'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_li_cases_raw['DateAdded']]
df_li_cases['name_li'] = clean_strings(df_li_cases['name_li'])
df_li_cases['abstract_li'] = clean_strings(df_li_cases['abstract_li'])
df_li_cases['DisplayTitle_li'] = clean_strings(df_li_cases['DisplayTitle_li'])
df_li_cases['court_li'] = clean_strings(df_li_cases['court_li'])
df_li_cases['DisplaySubtitle_li'] = clean_strings(df_li_cases['DisplaySubtitle_li'])
df_li_cases['subject_li'] = df_li_cases['subject_li'].apply(lambda x: set(x.strip("']['").split("', '")))

#### RS case citations

In [17]:
map_case_citations = {
    'source_ecli':'ecli',
    'source_paragraph':'source_paragraph',
    'target_ecli':'target_ecli',
    'target_paragraph':'target_paragraph'
}
df_rs_case_citations = df_rs_case_citations_raw[map_case_citations.keys()].rename(columns=map_case_citations)
df_rs_case_citations['doctype'] = 'c-citation-' + df_rs_case_citations['target_ecli'].map(str)

#### RS legislation citations

In [18]:
map_legislation_citations = {
    'source_ecli':'ecli',
    'source_paragraph':'source_paragraph',
    'target_article':'target_id',
    'target_article_paragraph':'target_paragraph',
    'target_article_webpage':'target_link'
}

df_rs_legislation_citations = df_rs_legislation_citations_raw[map_legislation_citations.keys()].rename(columns=map_legislation_citations)
df_rs_legislation_citations.loc[:,'target_name'] = None
df_rs_legislation_citations.loc[:,'target_sourcename'] = None
df_rs_legislation_citations['doctype'] = 'l-citation-' + df_rs_legislation_citations['target_id'].map(str)

#### RS case related decisions

In [14]:
map_case_related_decisions = {
    'case_id':'ecli',
    'related_cases':'referencing_case_ecli'
    }
df_rs_related_cases = df_rs_cases_raw.dropna(subset=['related_cases'])[['case_id', 'related_cases']]
df_rs_related_cases = df_rs_related_cases[map_case_related_decisions.keys()].rename(columns=map_case_related_decisions)
df_rs_related_cases.loc[:,'referenced_case_ecli'] = None
df_rs_related_cases['doctype'] = 'related-' + df_rs_related_cases['referencing_case_ecli'].map(str)
df_rs_related_cases['referencing_case_ecli'] = df_rs_related_cases['referencing_case_ecli'].apply(set)

#### Courts

In [12]:
df_court = pd.DataFrame()
courts_list = list(set(df_rs_cases_raw.authority.unique().tolist() + df_rs_opinions_raw.authority.unique().tolist() + df_li_cases_raw.authority.unique().tolist()))
courts_list = [i.replace('"','-') for i in courts_list]
courts_list.extend(['Other'])

df_court['name'] = clean_strings(courts_list)
df_court.loc[:,'type'] = None
df_court.loc[:,'level'] = None
df_court.loc[:,'country'] = None
df_court.loc[:,'language'] = None
df_court.loc[:,'jurisdiction'] = None
df_court.loc[:,'law_area'] = None
df_court.loc[:,'authority_level'] = None


#### Countries

In [13]:
df_country = df_country_raw.copy()
df_country.loc[:,'language'] = None
df_country.loc[:,'eea'] = 0
df_country = df_country[['country_id','name','language','flag','eu','eea']].rename(columns={'country_id':'id'})


#### Split data into RS only cases, LI only cases and RS+LI cases:

In [15]:
''' intersection of cases in li and case_raw '''
rsli_intersection = list(set(df_li_cases['ecli']).intersection(set(df_rs_cases['ecli'])))
rs_only = list(set(df_rs_cases['ecli']) - set(rsli_intersection))
li_only = list(set(df_li_cases['ecli']) - set(rsli_intersection))
print('number of li only cases: ', len(li_only))
print('number of rechtspraak only cases: ', len(rs_only))
print('number of overlapping cases: ', len(rsli_intersection))
print('number of rechtspraak total cases: ', len(df_rs_cases_raw))
print('number of li total cases: ', len(df_li_cases_raw))


number of li only cases:  44
number of rechtspraak only cases:  2631803
number of overlapping cases:  617
number of rechtspraak total cases:  2632420
number of li total cases:  661


In [16]:
### build separate dfs for li only and raw only cases
df_rs_only_cases = df_rs_cases[df_rs_cases['ecli'].isin(rs_only)]
df_li_only_cases = df_li_cases[df_li_cases['ecli'].isin(li_only)]


In [17]:
### merge overlapping li and raw cases into one df
df_rsli_intersection_cases = df_li_cases[df_li_cases['ecli'].isin(rsli_intersection)]
df_rsli_intersection_cases.set_index('ecli', inplace=True)
df_rs_cases.set_index('ecli', inplace=True)
df_rsli_intersection_cases = df_rsli_intersection_cases.join(df_rs_cases.drop('doctype', axis=1))
df_rs_cases.reset_index(inplace=True)
df_rsli_intersection_cases.reset_index(inplace=True)

In [40]:
print('Number of Cases in\nRS only cases: {}\nLI only cases: {}\nLI & RS cases: {}\nCase opinion: {}\nLegislation citation: {}\nCase citation: {}\nRelated decisions: {}'\
      .format(len(df_rs_only_cases), len(df_li_only_cases), len(df_rsli_intersection_cases), len(df_rs_opinions), len(df_rs_legislation_citations), len(df_rs_case_citations), len(df_rs_related_cases)))



NameError: name 'df_rs_legislation_citations' is not defined

---
## Data to DynamoDB

In [18]:
dynamodb = boto3.resource('dynamodb')  #, endpoint_url="http://localhost:8000")  # remove 'endpoint_url' to use web service

#### Table and key schema definition:

In [19]:
caselaw_table = dynamodb.create_table(
    TableName='Caselaw',
    KeySchema=[
        {
            'AttributeName': 'ecli',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'doctype',
            'KeyType': 'RANGE'  # Sort key
        },
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'ecli',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'doctype',
            'AttributeType': 'S'
        },
        {
            "AttributeName": "court",
            "AttributeType": "S"
        },
    ],
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'GSI-1',
            'KeySchema': [
                {
                    'AttributeName': 'doctype',
                    'KeyType': 'HASH'
                },
                                {
                    'AttributeName': 'court',
                    'KeyType': 'RANGE'
                },
            ],
            'Projection': {
                'ProjectionType': 'ALL',
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 10,
                'WriteCapacityUnits': 10
            },
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)
print("Table status:", caselaw_table.table_status)


Table status: CREATING


#### Functions to load data into table:

In [20]:
caselaw_table = dynamodb.Table('Caselaw')

def load_data(table, data, output=False):
    keys = [key['AttributeName'] for key in table.key_schema]
    counter = 0
    with table.batch_writer() as batch:
        #for item in data:
        for item in data.iterrows():
            item = item[1].to_dict()
            counter += 1
            if output:
                values = [item[key] for key in keys]
                print("Adding item:", dict(zip(keys, values)))
            #batch.put_item(Item=item, ConditionExpression='attribute_not_exists(ecli)')
            #batch.put_item(Item=item, ConditionExpression=Attr('ecli').not_exists())
            batch.put_item(Item=item)
    print(f'{counter} items added.')

def read_csv(filename, items):
    data = csv.DictReader(open(filename))
    for item in data:
        items.append(item)


In [ ]:
load_data(caselaw_table, df_rs_only_cases)

In [21]:
load_data(caselaw_table, df_li_only_cases)

44 items added.


In [22]:
load_data(caselaw_table, df_rsli_intersection_cases)

617 items added.


In [ ]:
load_data(caselaw_table, df_rs_opinions)

In [ ]:
load_data(caselaw_table, df_rs_case_citations)

In [ ]:
load_data(caselaw_table, df_rs_legislation_citations)

In [ ]:
load_data(caselaw_table, df_rs_related_cases)

In [62]:
load_data(caselaw_table, df_court)

ClientError: An error occurred (ValidationException) when calling the BatchWriteItem operation: One of the required keys was not given a value

In [ ]:
load_data(caselaw_table, df_country)

#### Example queries:

Uitspraak by ecli:

In [ ]:
pprint(caselaw_table.get_item(Key={'ecli': 'ECLI:NL:PHR:1996:AA1795',
                                   'doctype': 'uitspraak'}))

Opinion by ecli:

In [74]:
pprint(caselaw_table.query(
    ProjectionExpression="ecli, subject",
    KeyConditionExpression=Key('ecli').eq('ECLI:NL:PHR:2019:988') &
                           Key('doctype').eq('opinion'))  # [lc, cc, d]
)


AttributeError: 'Key' object has no attribute 'contains'

Case citations by ecli:

In [ ]:
pprint(caselaw_table.query(
    KeyConditionExpression=Key('ecli').eq('ECLI:NL:AGAMS:1934:1') &
                           Key('doctype').begins_with('c-citation'))  # [lc, cc, d]
)

Cases per subject and court

In [75]:
pprint(caselaw_table.query(
    IndexName='GSI-1',  # only needs to be specified if secondary index is used
    ProjectionExpression="ecli, court, country",
    FilterExpression='contains(#cl, :val)',
    ExpressionAttributeNames={'#cl': 'subject'},
    ExpressionAttributeValues={':val': 'Strafrecht'},
    KeyConditionExpression=Key('doctype').eq('opinion')) & Key('court').begins_with('A'))
)

{'Count': 2587,
 'Items': [{'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1968:AB6079'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1979:AC2094'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1980:AC6817'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1980:AC7058'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1983:AB7540'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1985:AB7743'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR:1985:AB7744'},
           {'country': 'NL',
            'court': 'Parket bij de Hoge Raad',
            'ecli': 'ECLI:NL:PHR

In [ ]:
''' usage example:

pprint(table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={"#yr": "year"},
    KeyConditionExpression=
            Key('year').eq(year) & Key('title').between(title_range[0], title_range[1]))
)
'''